In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt
from scipy.fftpack import * 

# 逐小时降水后处理
2021.12.23

并发地计算20年的4-8月逐小时降水累计结果，并放置到increment文件夹中

## 指定数据输入

In [2]:
import os
import glob
path_in                    = "/raid62/yycheng/MPAS/TPEMIP_92-25km/RCM_postprocess/RCM_merge_large/pre_hourly/accumulated/"
path_out                   = "/raid62/yycheng/MPAS/TPEMIP_92-25km/RCM_postprocess/RCM_merge_large/pre_hourly/increment/"  # 存放sys的临时文件

os.system("mkdir -p " + path_out)
files_in_list = []
files_out_list = []

multfilesin_names       = glob.glob(path_in + "merge_pre_????_RCM.nc")
multfilesout_names      = [path_out + os.path.basename(i) for i in multfilesin_names]

## 单个文件的处理函数

In [3]:
def write_temp_nc(file_in, file_out):
    print("----- dealing " + file_in)
    time_coords_name = 'Time'
    precip_vars_names = ['rainc','rainnc']
    ds_temp = xr.open_dataset(file_in)
    mod_pre = {}
    file_time = ds_temp[time_coords_name] # set time coords names
    for rainvar in precip_vars_names: # loop variables associated with precipitation and count increment, stroage for the forward time
        preforward = ds_temp[rainvar][1:,:,:].assign_coords({time_coords_name:ds_temp[rainvar][0:-1,:,:][time_coords_name]})
        preback    = ds_temp[rainvar][0:-1,:,:]
        mod_pre[rainvar] = preforward - preback
    ds_out = xr.Dataset(mod_pre)
    ds_out.to_netcdf(file_out)
    ds_out.close()

## 并发处理每个文件

In [4]:
# 10mins for 20years hourly precipitation.....

from multiprocessing import Pool
import os, time, random
print('Parent process %s.' % os.getpid())
p = Pool(5)
for i in range(len(multfilesin_names)):
    p.apply_async(write_temp_nc, args=( (multfilesin_names[i], multfilesout_names[i]) ) )
print('Waiting for all subprocesses done...')
p.close()
p.join()
print('All subprocesses done.')

Parent process 8279.
----- dealing /raid62/yycheng/MPAS/TPEMIP_92-25km/RCM_postprocess/RCM_merge_large/pre_hourly/accumulated/merge_pre_1998_RCM.nc
----- dealing /raid62/yycheng/MPAS/TPEMIP_92-25km/RCM_postprocess/RCM_merge_large/pre_hourly/accumulated/merge_pre_1999_RCM.nc
----- dealing /raid62/yycheng/MPAS/TPEMIP_92-25km/RCM_postprocess/RCM_merge_large/pre_hourly/accumulated/merge_pre_2000_RCM.nc
----- dealing /raid62/yycheng/MPAS/TPEMIP_92-25km/RCM_postprocess/RCM_merge_large/pre_hourly/accumulated/merge_pre_2001_RCM.nc
----- dealing /raid62/yycheng/MPAS/TPEMIP_92-25km/RCM_postprocess/RCM_merge_large/pre_hourly/accumulated/merge_pre_2002_RCM.nc
Waiting for all subprocesses done...
----- dealing /raid62/yycheng/MPAS/TPEMIP_92-25km/RCM_postprocess/RCM_merge_large/pre_hourly/accumulated/merge_pre_2003_RCM.nc
----- dealing /raid62/yycheng/MPAS/TPEMIP_92-25km/RCM_postprocess/RCM_merge_large/pre_hourly/accumulated/merge_pre_2004_RCM.nc
----- dealing /raid62/yycheng/MPAS/TPEMIP_92-25km/RCM